In [36]:
patch_dict={'op': '<=',
  'opr': [{'state': 'input', 'type': 'int', 'name': 'nick', 'coord': "infer.c:35:27"},
   {'state': 'element',
    'type': 'int',
    'name': '_const',
    'coord': None,
    'const': 5}]}

In [37]:
from instrumentation import *


In [38]:
'''
patch synthesis
insert tmp var heapobject
insert patch (return location)

'''

'\npatch synthesis\ninsert tmp var heapobject\ninsert patch (return location)\n\n'

In [58]:
filelist=read_file("infer.c")

In [46]:
def insert_tmp_var(filelist,patch_dict):
    if patch_dict["opr"]=="1":
        return filelist,None
    patchlist=[]
    for opr in patch_dict["opr"]:
        if opr["name"]=="_const":
            patchlist.append(str(opr["const"]))
        else:
            line=int(opr["coord"].split(":")[1])
            state=opr["state"]
            insert_stat=f"int tmp_{opr['name']} = {opr['name']};\n"
            patchlist.append("tmp_"+opr["name"])
            if state=="var":
                filelist[line-1]=filelist[line-1][:-1]+insert_stat
            elif state=="input":
                tmp=line-1
                while filelist[tmp][-2]!="{":
                    tmp+=1
                filelist[tmp]=filelist[tmp][:-1]+insert_stat
    
    return filelist,patchlist[0]+patch_dict["op"]+patchlist[1]
            
def insert_heap_object(filelist,o,otype,line):
    insert_stat=f"{otype} tmp_{o} = {o};\n"
    filelist[line-1]=filelist[line-1][:-1]+insert_stat
    return filelist,"tmp_"+o

def cur_patch(patch,o):
    if patch:
        return f"if({patch})free({o});"
    else: 
        return f"free({o});"
    

def insert_patch(filelist,line,patch):
    filelist[line-1]=patch+filelist[line-1]
    return filelist
    

In [57]:
filelist

['typedef struct _p11_dict {\n',
 '    struct p11_dictbucket **buckets;\n',
 '}p11_dict;\n',
 '\n',
 'typedef struct p11_dictbucket {\n',
 '    struct p11_dictbucket *next;\n',
 '} dictbucket;\n',
 '\n',
 'p11_dict *\n',
 'p11_dict_new (){\n',
 '    p11_dict *dict;**(*p)->a.b=b+c;\n',
 '    dict = malloc (sizeof (p11_dict));\n',
 '    if (dict) {\n',
 '        dict->buckets = (dictbucket **)malloc (sizeof (dictbucket *));\n',
 '        if (!dict->buckets) {\n',
 '            free (dict);\n',
 '            return NULL;\n',
 '        }\n',
 '    }\n',
 '    return dict;\n',
 '}\n',
 '\n',
 '\n',
 'p11_dict *\n',
 'func(int c){\n',
 '    p11_dict *dict;\n',
 '    dict =p11_dict_new();\n',
 '    if(c==0){\n',
 '        return NULL;\n',
 '    }\n',
 '    return dict;\n',
 '}\n',
 '\n',
 'p11_dict *\n',
 'p11_constant_reverse (int nick)\n',
 '{int tmp_nick = nick;\n',
 '    p11_dict *lookups;\n',
 '    lookups = p11_dict_new ();\n',
 '    if (nick) {\n',
 '        return NULL; p11_dict * tmp

In [59]:
fl,pd=insert_tmp_var(filelist,patch_dict)

In [60]:
insert_heap_object(fl,"lookups"," p11_dict *",38)

(['typedef struct _p11_dict {\n',
  '    struct p11_dictbucket **buckets;\n',
  '}p11_dict;\n',
  '\n',
  'typedef struct p11_dictbucket {\n',
  '    struct p11_dictbucket *next;\n',
  '} dictbucket;\n',
  '\n',
  'p11_dict *\n',
  'p11_dict_new (){\n',
  '    p11_dict *dict;**(*p)->a.b=b+c;\n',
  '    dict = malloc (sizeof (p11_dict));\n',
  '    if (dict) {\n',
  '        dict->buckets = (dictbucket **)malloc (sizeof (dictbucket *));\n',
  '        if (!dict->buckets) {\n',
  '            free (dict);\n',
  '            return NULL;\n',
  '        }\n',
  '    }\n',
  '    return dict;\n',
  '}\n',
  '\n',
  '\n',
  'p11_dict *\n',
  'func(int c){\n',
  '    p11_dict *dict;\n',
  '    dict =p11_dict_new();\n',
  '    if(c==0){\n',
  '        return NULL;\n',
  '    }\n',
  '    return dict;\n',
  '}\n',
  '\n',
  'p11_dict *\n',
  'p11_constant_reverse (int nick)\n',
  '{int tmp_nick = nick;\n',
  '    p11_dict *lookups;\n',
  '    lookups = p11_dict_new (); p11_dict * tmp_lookups = 

In [62]:
patch=cur_patch(pd,"lookups")

In [63]:
insert_patch(filelist,40,patch)

['typedef struct _p11_dict {\n',
 '    struct p11_dictbucket **buckets;\n',
 '}p11_dict;\n',
 '\n',
 'typedef struct p11_dictbucket {\n',
 '    struct p11_dictbucket *next;\n',
 '} dictbucket;\n',
 '\n',
 'p11_dict *\n',
 'p11_dict_new (){\n',
 '    p11_dict *dict;**(*p)->a.b=b+c;\n',
 '    dict = malloc (sizeof (p11_dict));\n',
 '    if (dict) {\n',
 '        dict->buckets = (dictbucket **)malloc (sizeof (dictbucket *));\n',
 '        if (!dict->buckets) {\n',
 '            free (dict);\n',
 '            return NULL;\n',
 '        }\n',
 '    }\n',
 '    return dict;\n',
 '}\n',
 '\n',
 '\n',
 'p11_dict *\n',
 'func(int c){\n',
 '    p11_dict *dict;\n',
 '    dict =p11_dict_new();\n',
 '    if(c==0){\n',
 '        return NULL;\n',
 '    }\n',
 '    return dict;\n',
 '}\n',
 '\n',
 'p11_dict *\n',
 'p11_constant_reverse (int nick)\n',
 '{int tmp_nick = nick;\n',
 '    p11_dict *lookups;\n',
 '    lookups = p11_dict_new (); p11_dict * tmp_lookups = lookups;\n',
 '    if (nick) {\n',
 'i